## Packages 

In [ ]:
import pandas as pd
import numpy as np
from newsapi import NewsApiClient
from dateutil.parser import parse
from collections import Counter
import pickle

# Creating NewsApi Client

In [ ]:
api = NewsApiClient(api_key='XXXXXXXXXXXXXXXXXXXXXXXXXXXX')

## Fetching News from get_everything

In [ ]:
top_Headline = api.get_everything(q='infosys',from_param='2020-02-25',to ='2020-03-03',language='en',page_size=100)

In [ ]:
Articles = top_Headline['articles']  # Articles 

# Sentiment Analysis

In [ ]:
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer
#nltk.download('vader_lexicon')

In [ ]:
sia = SentimentIntensityAnalyzer()

## Changing Vader Lexicon to Financial Lexicon

Refered Jason Yip TradeAlgo for Vader Financial Lexicon 

In [ ]:
import csv
import pandas as pd

 # stock market lexicon
stock_lex = pd.read_csv('lexicon_data/stock_lex.csv')
stock_lex['sentiment'] = (stock_lex['Aff_Score'] + stock_lex['Neg_Score'])/2
stock_lex = dict(zip(stock_lex.Item, stock_lex.sentiment))
stock_lex = {k:v for k,v in stock_lex.items() if len(k.split(' '))==1}
stock_lex_scaled = {}
for k, v in stock_lex.items():
    if v > 0:
        stock_lex_scaled[k] = v / max(stock_lex.values()) * 4
    else:
        stock_lex_scaled[k] = v / min(stock_lex.values()) * -4

 # Loughran and McDonald
    positive = []
    with open('lexicon_data/lm_positive.csv', 'r') as f:
        reader = csv.reader(f)
        for row in reader:
            positive.append(row[0].strip())
    
    negative = []
    with open('lexicon_data/lm_negative.csv', 'r') as f:
        reader = csv.reader(f)
        for row in reader:
            entry = row[0].strip().split(" ")
            if len(entry) > 1:
                negative.extend(entry)
            else:
                negative.append(entry[0])

final_lex = {}
final_lex.update({word:2.0 for word in positive})
final_lex.update({word:-2.0 for word in negative})
final_lex.update(stock_lex_scaled)
final_lex.update(sia.lexicon)
sia.lexicon = final_lex

In [ ]:
def get_news_sentiment(news): 
    # create Vader object of passed news text 
    sia = SentimentIntensityAnalyzer() 
    # set sentiment 
    if sia.polarity_scores(news)['compound'] >= 0.05 : 
        return 'Positive'  
    elif sia.polarity_scores(news)['compound'] <= - 0.05 : 
        return 'Negative' 
    else : 
        return "Neutral"

In [ ]:
data = {}
for date in dates:
    li = []
    for article in Articles:
        if (parse(article.get('publishedAt')).strftime('%Y-%m-%d') == str(date.date())) and (article.get('source').get('name') in ['Moneycontrol.com','Indianexpress.com','The Times of India','Business-standard.com']):
            li.append(get_news_sentiment(article['description']))
    
    data[date.date().strftime('%Y-%m-%d')]=Counter(li).most_common(1)

In [ ]:
data = pd.DataFrame(data).transpose()

In [ ]:
data